In [6]:
%reset -f
%matplotlib qt

##################################################
## A320-214 Engine Parameter Characterization
##
## First attempt on a multivariate polynomial
## regression algorithm to predict engine TSFC
## as a function of Mach, Altitude and Thrust
##
##################################################
## Author: J.M. Roldan
## Date: 11/13/2020
## License: GPL 3.0
## Discord: TazX [Z+1]#0405
## Status: Final
##################################################

import pandas as pd
from sklearn import preprocessing
from sklearn import linear_model
import matplotlib.pyplot as plt
import seaborn as sns

################
# Figure Set-up
################

a4_dims = (11.7, 8.27)
sns.set(font_scale=0.5)
fig, ax =plt.subplots(3,2, figsize=a4_dims)
ax[0][0].set(title='Take-off', xlabel='Mach', ylabel='TSFC')
ax[0][1].set(title='Climb', xlabel='Mach', ylabel='TSFC')
ax[1][0].set(title='Cruise', xlabel='Mach', ylabel='TSFC')
ax[1][1].set(title='Descent', xlabel='Mach', ylabel='TSFC')
ax[2][0].set(title='Hold', xlabel='Mach', ylabel='TSFC')

################
# Data Init
################

for phase in range(5):
    # CSV read and tiddy-up
    df = pd.read_csv("../a320-214.csv")
    df = df.rename(columns = {'ALT.':'Alt', 'MACH':'Mach'}, inplace=False)

    if (phase == 0):
        df = df[df['PHASE'] == 'TO']
        i = 0
        j = 0
    elif (phase == 1):
        df = df[df['PHASE'] == 'CLB']
        i = 0
        j = 1
    elif (phase == 2):
        df = df[df['PHASE'] == 'CRZ']
        i = 1
        j = 0
    elif (phase == 3):
        df = df[df['PHASE'] == 'DSC']
        i = 1
        j = 1
    else:
        df = df[df['PHASE'] == 'HLD']
        i = 2
        j = 0
        
    df = df[df['Mach'] != 0]
    df = df.reset_index(drop=True)

    # Define Independent and Dependent Variables
    X = df[['Mach', 'Alt', 'FN']]
    y = df['TSFC']

    ######################################################
    # Fit and predict with Multiple Polynomial Regression
    ######################################################

    #generate a model of polynomial features
    poly = preprocessing.PolynomialFeatures(degree=3)

    #transform the x data for proper fitting (for single variable type it returns,[1,x,x**2])
    X_poly = poly.fit_transform(X)

    #generate the regression object
    regr = linear_model.LinearRegression()

    #preform the actual regression
    regr.fit(X_poly, y)

    #some statistics
    r_sq = regr.score(X_poly,y)
    b0 = regr.intercept_

    #new DF and check with original table
    columns = ['Mach', 'Alt', 'FN', 'TSFC_orig', 'TSFC_pred', 'error']
    new_df = pd.DataFrame(columns=columns)

    #prediction loop
    for index in range(len(df.index)):
        predict_ = poly.fit_transform(([[df['Mach'][index], df['Alt'][index], df['FN'][index]]]))
        predCFN = regr.predict(predict_)
        sq_error = abs(predCFN[0]-df['TSFC'][index])**2
        new_df = new_df.append({'Mach':df['Mach'][index],\
                                'Alt':df['Alt'][index],\
                                'FN':df['FN'][index],\
                                'TSFC_orig':df['TSFC'][index],\
                                'TSFC_pred':predCFN[0],\
                                'error':sq_error}, ignore_index = True)

    mse = sum(new_df['error'])/len(new_df.index)
    print("R2 score: %.6f" % r_sq)
    print("MSE: %.6f" % mse)
    print(phase)
    
    #Plot results
    sns.set_theme(style="darkgrid")
    
    sns.scatterplot(data=new_df, x="Mach", y="TSFC_orig", s=8, ax = ax[i][j])
    sns.scatterplot(data=new_df, x="Mach", y="TSFC_pred", s=8, ax = ax[i][j])



R2 score: 0.997347
MSE: 0.000010
0
R2 score: 0.999988
MSE: 0.000000
1
R2 score: 0.987612
MSE: 0.000014
2
R2 score: 0.168761
MSE: 85.268686
3
R2 score: 0.996541
MSE: 0.000002
4
